In [49]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.metrics import accuracy_score

In [2]:
#define constants
#unrolled through 28 time steps
time_steps=100
#hidden LSTM units
num_units=128
#rows of 28 pixels
n_input=12
#learning rate for adam
learning_rate=0.001
#mnist is meant to be classified in 10 classes(0-9).
n_classes=6
#size of batch
batch_size=120

In [3]:
train_data = pd.read_csv("train_data_400.csv").dropna()
test_data = pd.read_csv("test_data.csv")

In [4]:
# set(test_data["type_1"])

In [5]:
# train_data["type_1"] = train_data["user_id"].map(str) +"_"+ train_data["action_id"]

# Data Preprocessing

In [6]:
# dataTemp : CSV input/out 섞여있는 데이터 
# period : 자를 timeSteps
# onehot : default True, 원핫으로 return 할지 말지 결정
# return : 다 만들어진 X , Y
def dataPreprocessing(dataTemp , period , onehot = True , dropCol = True ):
    encoding = {'jog': 0, 'dws': 1, 'ups': 2, 'wlk': 3, 'sit': 4, 'std': 5}
    retY = pd.Series([])
#     retY = np.array([])
    retX = dataTemp
    temptype = -1
    cnt = 0
    dataTemp["type_1"] = dataTemp["user_id"].map(str) +"_"+ dataTemp["action_id"]
    for dataIter in range(dataTemp["action_id"].size//period):
        retY[cnt] = encoding[dataTemp["action_id"].iloc[dataIter*period][:3]]
        comp = dataTemp["type_1"].iloc[dataIter*period]
        if comp == temptype:
            cnt += 1
        else:
#             print(cnt)
            temptype = comp
            retX = retX.drop(retX.index[dataIter:dataIter+period])
    if onehot:
        retY = pd.get_dummies(retY)
    if dropCol:
        retX = retX.drop(["action_id"] , axis = 1)
        retX = retX.drop(["user_id"] , axis = 1)
        retX = retX.drop(["time_50hz"] , axis = 1)
        retX = retX.drop(["type_1"] , axis = 1)
    return retX , retY
        

In [7]:
train_X , train_Y = dataPreprocessing(train_data , time_steps , True)
test_X , test_Y = dataPreprocessing(test_data , time_steps , True)

In [8]:
train_X.shape , train_Y.shape , test_X.shape , test_Y.shape

((1209003, 12), (12090, 6), (167862, 12), (1678, 6))

In [9]:
# encoding = {'jog': 0, 'dws': 1, 'ups': 2, 'wlk': 3, 'sit': 4, 'std': 5}
# y_train = train_data["action_id"].apply(lambda x:encoding[x[:3]])
# y_test= test_data["action_id"].apply(lambda x:encoding[x[:3]])

In [10]:
# y_train = pd.get_dummies(y_train)
# y_test = pd.get_dummies(y_test)

In [11]:
# temp_input = train_data.drop(["action_id"] , axis = 1)
# temp_input = temp_input.drop(["user_id"] , axis = 1)
# temp_input = temp_input.drop(["time_50hz"] , axis = 1)
# test_input = test_data.drop(["action_id"] , axis = 1)
# test_input = test_input.drop(["user_id"] , axis = 1)
# test_input = test_input.drop(["time_50hz"] , axis = 1)

In [12]:
# temp_X = temp_input[:1240503]
# temp_Y = y_train[:1240503]
# test_X = test_input[:170000]
# test_Y = y_test[:170000]

In [13]:
#weights and biases of appropriate shape to accomplish above task
out_weights=tf.Variable(tf.random_normal([num_units,n_classes]))
out_bias=tf.Variable(tf.random_normal([n_classes]))

#defining placeholders
#input image placeholder
x=tf.placeholder(tf.float32 ,[None,time_steps , n_input])
#input label placeholder
y=tf.placeholder(tf.int32 ,[None,n_classes])

In [14]:
input=tf.unstack(x ,time_steps,1)

In [15]:
#defining the network
lstm_layer=rnn.BasicLSTMCell(num_units,forget_bias=1)
outputs,_=rnn.static_rnn(lstm_layer,input,dtype="float32")

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


In [16]:
#converting last output of dimension [batch_size,num_units] to [batch_size,n_classes] by out_weight multiplication
prediction=tf.matmul(outputs[-1],out_weights)+out_bias

In [17]:
Yhat = tf.argmax(prediction,1)

In [18]:
#loss_function
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction,labels=y))
#optimization
opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

#model evaluation
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [19]:
#initialize variables
init=tf.global_variables_initializer()
test_X_temp = np.array(test_X[:160000]).reshape(-1,time_steps , n_input)
epoch = 20
sess = tf.Session()

sess.run(init)
for ep  in range(1,epoch+1):    
    iter=0
    while iter<99:
        batch_x = train_X[iter*batch_size*time_steps:(iter+1)*batch_size*time_steps]
        batch_y = train_Y[iter*batch_size:(iter+1)*batch_size]
        batch_x = np.array(batch_x)
        batch_x=batch_x.reshape((batch_size,time_steps,n_input))
        sess.run(opt, feed_dict={x: batch_x, y: batch_y})  
#             if iter % 10 == 0:
#                 acc=sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
#                 los=sess.run(loss,feed_dict={x:batch_x,y:batch_y})
        iter=iter+1
    if ep % 1 == 0:            
        test_acc = sess.run(accuracy, feed_dict={x: test_X_temp, y: test_Y[:1600]})
        test_los = sess.run(loss, feed_dict={x: test_X_temp, y: test_Y[:1600]})
        print("Epoch {} ,  Test Acc {} , los {} ".format(ep,test_acc , test_los))


Epoch 1 ,  Test Acc 0.27937498688697815 , los 6.747722625732422 
Epoch 2 ,  Test Acc 0.27687498927116394 , los 4.385247230529785 
Epoch 3 ,  Test Acc 0.2787500023841858 , los 4.353761196136475 
Epoch 4 ,  Test Acc 0.33812499046325684 , los 2.7773358821868896 
Epoch 5 ,  Test Acc 0.35062500834465027 , los 2.194559097290039 
Epoch 6 ,  Test Acc 0.41999998688697815 , los 2.0462234020233154 
Epoch 7 ,  Test Acc 0.47062501311302185 , los 1.6268396377563477 
Epoch 8 ,  Test Acc 0.5099999904632568 , los 1.4484905004501343 
Epoch 9 ,  Test Acc 0.6162499785423279 , los 1.3106616735458374 
Epoch 10 ,  Test Acc 0.6462500095367432 , los 1.2842544317245483 
Epoch 11 ,  Test Acc 0.5181249976158142 , los 1.483980655670166 
Epoch 12 ,  Test Acc 0.6612499952316284 , los 1.2103365659713745 
Epoch 13 ,  Test Acc 0.659375011920929 , los 1.2302844524383545 
Epoch 14 ,  Test Acc 0.6631249785423279 , los 1.1532647609710693 
Epoch 15 ,  Test Acc 0.6812499761581421 , los 1.1384409666061401 
Epoch 16 ,  Test Ac

In [20]:
# test_data_eval 에 분류를 위한 type_1 을 넣는데.
test_data_eval = test_data
test_data_eval["type_1"] = test_data["user_id"].map(str) +"_"+ test_data["action_id"]

In [21]:
# type 의 종류를 가지고 와서 나중에 loop 를 돌면서 accuracy 를 만들것임
dic_testType = list(set(test_data_eval["type_1"]))
# dic_testType

In [22]:
# full loop 만들어보기


# 한 타입만 일단 가져와보고
tempX = test_data_eval[(test_data_eval["type_1"] == '2_wlk_15')]
# matrix 에 맞게 뒤에 잘라내줌.
tempX = tempX[:(tempX.shape[0] // time_steps)*time_steps]
# 여기에서 데이터를 X,Y 로 먼저 나눠준다. onehot 해뒀고 필요없는 column 까지 다 Drop
evalTestX , evalTestY = dataPreprocessing(tempX , time_steps , True , True)
evalTestX = np.array(evalTestX).reshape(-1,time_steps , n_input)
# one hot 에 하나만 있기때문에 나머지 다 넣어주고 순서 원래대로 변경
#         for i in range(6):
#             if i not in set(evalTestY):
#                 evalTestY[i] = 0
#         evalTestY=evalTestY[[0,1,2,3,4,5]]
print('predict : ' , sess.run(Yhat , feed_dict={x:evalTestX}) , " Real Y : " , evalTestY.columns)
#     print(evalTestX.shape , evalTestY.shape , ite)

predict :  [3 3 3 3 2 3 3 3 2 3 3 3 3 3 3 3]  Real Y :  Int64Index([3], dtype='int64')


In [54]:
# full loop 만들어보기
y_pred = []
y_true = []
# >>> accuracy_score(y_true, y_pred)
for ite in dic_testType:
    # 한 타입만 일단 가져와보고
    tempX = test_data_eval[(test_data_eval["type_1"] == ite)]
    # matrix 에 맞게 뒤에 잘라내줌.
    tempX = tempX[:(tempX.shape[0] // time_steps)*time_steps]
    # 여기에서 데이터를 X,Y 로 먼저 나눠준다. onehot 해뒀고 필요없는 column 까지 다 Drop
    evalTestX , evalTestY = dataPreprocessing(tempX , time_steps , True , True)
    evalTestX = np.array(evalTestX).reshape(-1,time_steps , n_input)
    # one hot 에 하나만 있기때문에 나머지 다 넣어주고 순서 원래대로 변경
#         for i in range(6):
#             if i not in set(evalTestY):
#                 evalTestY[i] = 0
#         evalTestY=evalTestY[[0,1,2,3,4,5]]
    yh = sess.run(Yhat , feed_dict={x:evalTestX})
    pre = Counter(yh).most_common(1)[0][0]
    tru = evalTestY.columns[0]
    y_pred.append(pre)
    y_true.append(tru)
    print("ite : " , ite , ' predict : ' , yh , " Real Y : " , tru)
#     print('predict : ' , pre , " Real Y : " , tru)
#     print(evalTestX.shape , evalTestY.shape , ite)}

ite :  1_wlk_8  predict :  [3 3 0 3 2 3 3 3 3 3 3 2 3 2 3 1 2 3 2 3 3 3 3 3 2 2 3 3 3 3 3 3 2 3 3 3 3]  Real Y :  3
ite :  1_sit_5  predict :  [4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]  Real Y :  4
ite :  2_jog_9  predict :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3]  Real Y :  0
ite :  3_wlk_15  predict :  [3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]  Real Y :  3
ite :  3_wlk_8  predict :  [3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]  Real Y :  3
ite :  3_ups_4  predict :  [3 2 2 3 3 3 3 3 3 3 3 2 3 3 3 3 3 2 2 2 3 3 3 3 3 3 2 3 2 3 3 3 3 3 3 3]  Real Y :  2
ite :  1_dws_11  predict :  [5 3 3 2 1]  Real Y :  1
ite :  2_jog_16  predict :  [0 0 0 0 0 0 0 0 0]  Real Y :  0
ite :  2_ups_12  predict :  [0 3]  Real Y : 

In [52]:
accuracy_score(y_true, y_pred)

0.6

In [ ]:

temp = test_data_eval[(test_data_eval["type_1"] == '2_dws_2')]
# reshape 이 자동으로 되게 time_steps 로 잘라줌
temp = temp[:(temp.shape[0] // time_steps)*time_steps]
xx , yy = dataPreprocessing(temp , time_steps , True , True)
xx = np.array(xx).reshape(-1,time_steps , n_input)
with tf.Session() as sess:
    sess.run(init)
    test_acc = sess.run(accuracy, feed_dict={x: xx, y: yy})
    test_los = sess.run(loss, feed_dict={x: xx, y: yy})

In [ ]:
for i in inputDic:
    temp = test_data_eval[(test_data_eval["type_1"] == i)]
    # reshape 이 자동으로 되게 time_steps 로 잘라줌
    temp = temp[:(temp.shape[0] // time_steps)*time_steps]
    xx , yy = dataPreprocessing(temp , time_steps , True , True)
    xx = np.array(xx).reshape(-1,time_steps , n_input)
    

In [ ]:
test_X_temp = np.array(test_X[:160000]).reshape(-1,time_steps , n_input)
with tf.Session() as sess:
    sess.run(init)
    test_acc = sess.run(accuracy, feed_dict={x: test_X_temp, y: test_Y[:1600]})
    test_los = sess.run(loss, feed_dict={x: test_X_temp, y: test_Y[:1600]})